Загрузка модели и датасета

In [4]:
!pip install annoy

  Using cached annoy-1.17.3-cp39-cp39-win_amd64.whl


DEPRECATION: vtk -PKG-VERSION has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of vtk or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: Could not install packages due to an OSError: [WinError 32] Процесс не может получить доступ к файлу, так как этот файл занят другим процессом: 'D:\\Anaconda\\envs\\uv_net\\Lib\\site-packages\\annoy\\__init__.py'
Consider using the `--user` option or check the permissions.



In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

import pathlib
import string
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree

from base import BaseDataset
from models import Segmentation

import methods

class AttrDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [2]:
def sample_from_letter(fnm_list, n_items, case = None):
    #case: ('lower','upper')
    df = pd.DataFrame()
    df['fname'] = [fnm.split('.')[0] for fnm in fnm_list]
    spl = df.fname.str.split('_')
    df['letter'] = spl.apply(lambda x: x[0])
    df['case'] = spl.apply(lambda x: x[-1])
    df['class'] = df['letter']+df['case']
    df['class'] = pd.Categorical(df['class'])
    df['label'] = df['class'].cat.codes    

    n_classes = df['class'].nunique()
    
    if case is not None:
        df = df[df.case == case]
    samples = df.groupby('label').apply(lambda x: x.sample(n_items))
    fnm_labels = samples[['fname','label']].set_index('fname').label.to_dict()
    return n_classes, fnm_labels

class RankingDataset(BaseDataset):
    @staticmethod
    def num_classes():
        return self.num_classes

    def __init__(
        self,
        root_dir,
        fnm_labels,
        num_classes,
        _center_and_scale=True,
        random_rotate=False,
    ):
        """
        Args:
            _center_and_scale (bool, optional): Whether to center and scale the solid. Defaults to True.
            random_rotate (bool, optional): Whether to apply random rotations to the solid in 90 degree increments. Defaults to False.
        """
        # path = pathlib.Path(root_dir)
        self.random_rotate = random_rotate
        self.num_classes = num_classes
        
        self.lbs = fnm_labels

        file_paths = [pathlib.Path(root_dir+fnm+'.bin') for fnm in fnm_labels.keys()]
        print(file_paths[0], file_paths[0].exists())
        self.load_graphs(file_paths, _center_and_scale)
        print("Done loading {} files".format(len(self.data)))

    def load_one_graph(self, file_path):
        # Load the graph using base class method
        sample = super().load_one_graph(file_path)
        # Additionally get the label from the filename and store it in the sample dict

        sample["label"] = torch.tensor([self.lbs[str(file_path.stem)]]).long()
        return sample

    def _collate(self, batch):
        collated = super()._collate(batch)
        collated["label"] =  torch.cat([x["label"] for x in batch], dim=0)
        return collated

In [3]:
args = AttrDict({})
args.batch_size = 128
args.random_rotate = False
args.num_workers = 0
args.checkpoint = 'D:/NIR/results/segmentation100/0925/222631/best.ckpt'
device = torch.device('cpu:0')

model = Segmentation.load_from_checkpoint(args.checkpoint).model.to(device = device)

D:\Anaconda\Lib\site-packages\deprecate\deprecation.py:115: FutureWarning: The `IoU` was deprecated since v0.7 in favor of `torchmetrics.classification.jaccard.JaccardIndex`. It will be removed in v0.8.
  stream(template_mgs % msg_args)


In [4]:
fnm_list = os.listdir('D:/NIR/SolidLetters/graph_with_eattr')
N_ITEMS_PER_CLASS = 100

# creating loaders for SolidLetters dataset quering
test_loaders = []
for case in ('lower', 'upper'):
    ncl, fnm_labels = sample_from_letter(fnm_list, N_ITEMS_PER_CLASS, case)
    dset = RankingDataset('D:/NIR/SolidLetters/graph_with_eattr/', 
                           fnm_labels, 
                           ncl)
    test_loaders.append(dset.get_dataloader(batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers))

D:\NIR\SolidLetters\graph_with_eattr\a_Squada One_lower.bin True


  0%|                                                                                         | 0/2600 [00:00<?, ?it/s]D:\Anaconda\Lib\site-packages\dgl\data\graph_serialize.py:189: DGLWarning: You are loading a graph file saved by old version of dgl.              Please consider saving it again with the current format.
  dgl_warning(
100%|██████████████████████████████████████████████████████████████████████████████| 2600/2600 [00:28<00:00, 89.72it/s]


Done loading 2600 files
D:\NIR\SolidLetters\graph_with_eattr\a_Piedra_upper.bin True


  0%|                                                                                         | 0/2600 [00:00<?, ?it/s]D:\Anaconda\Lib\site-packages\dgl\data\graph_serialize.py:189: DGLWarning: You are loading a graph file saved by old version of dgl.              Please consider saving it again with the current format.
  dgl_warning(
100%|██████████████████████████████████████████████████████████████████████████████| 2600/2600 [00:31<00:00, 82.59it/s]


Done loading 2600 files


Получение векторных представлений

In [5]:
def encode(model, loader, device):
    embs_list = []
    labels_list = []
    with torch.no_grad():  
        for batch in loader:
            inputs = batch["graph"].to(device)
            inputs.ndata["x"] = inputs.ndata["x"].permute(0, 3, 1, 2)
            inputs.edata["x"] = inputs.edata["x"].permute(0, 2, 1)
            embs_list.append(model.encode_part(inputs).to(device=torch.device('cpu'))) 
            labels_list.append(batch["label"].to(device=torch.device('cpu')))
    return embs_list, labels_list
#переделать (возвращает только для части букв)
def get_embs(test_loaders, model, device):
    model = model.eval()
    metr = []
    for loader in test_loaders:
        e_list, l_list = encode(model, loader, device)
        embs = torch.cat(e_list,dim=0).numpy()
        lbs = torch.cat(l_list,dim=0).numpy()
    return embs, lbs

Поиск похожих

In [6]:
class SearchSimilarModels:
    def __init__(self):
        pass
    
    def cals_map_all(self, test_loaders, model, device):
        model = model.eval()
        metr = []
        for loader in test_loaders:
            e_list, l_list = encode(model, loader, device)
            embs = torch.cat(e_list,dim=0).numpy()
            lbs = torch.cat(l_list,dim=0).numpy()
            metr.append(calc_map(embs, lbs))
        return np.mean(metr)

    def calc_map(self, X, labels, K = 5):
#         tree = KDTree(X, leaf_size=40)  # creating kd tree
#         _, ind = tree.query(X, k=K+1)  # quering nearest items

        is_valid_label = (labels[ind[:,1:]] == labels.reshape(-1,1)).astype(int)

        cum_sum = np.cumsum(is_valid_label, axis=1)
        P_K = cum_sum/np.arange(1, K+1).reshape(1,-1)
        AP_K = P_K.sum(axis=1) / np.clip(cum_sum[:,-1],1, K)

        return AP_K.mean()

In [7]:
dataset, label = get_embs(test_loaders, model, device)

In [8]:
K=5

### Sk learn KD-Tree

In [9]:
tree = KDTree(dataset, leaf_size=40)  # creating kd tree
_, ind = tree.query(dataset, k=K+1)# quering nearest items
search = SearchSimilarModels()
search.calc_map(ind, label)

0.7365597873263889

### KD-Tree with cosine distance

In [37]:
#задавать число соседей самим в кд три
cosine_tree = methods.KDTree(dataset)
ind = cosine_tree.search(dataset)
search.calc_map(ind, label)

0.6659326171875

### KNN with cosine distance

In [14]:
cosine_knn = methods.DistributedCosineKnn(K)
ind, _ = cosine_knn.fit(dataset)
search.calc_map(ind, label)

0.7525330946180555

### HNSW

In [15]:
hnsw = methods.HNSW(dataset)
ind = hnsw.search(dataset, dataset, K+1)
search.calc_map(ind, label)

0.7383924696180555

### Понижение размерности PCA

In [16]:
from sklearn.decomposition import PCA

In [17]:
transformer = PCA(n_components=100)
pca_data = transformer.fit_transform(dataset)

In [18]:
tree = KDTree(pca_data, leaf_size=40)  # creating kd tree
_, ind = tree.query(pca_data, k=K+1)# quering nearest items
search = SearchSimilarModels()
search.calc_map(ind, label)

0.7357188585069444

In [20]:
cosine_tree = methods.KDTree(pca_data, 0)
ind = cosine_tree.search(pca_data)
search.calc_map(ind, label)

0.6768950737847221

In [21]:
cosine_knn = methods.DistributedCosineKnn(K)
ind, _ = cosine_knn.fit(pca_data)
search.calc_map(ind, label)

0.7376969401041666

In [22]:
hnsw = methods.HNSW(pca_data)
ind = hnsw.search(pca_data, pca_data, K+1)
search.calc_map(ind, label)

0.732473415798611

### Понижение размерности TSNE

In [23]:
from sklearn.manifold import TSNE

In [31]:
X_embedded = TSNE(n_components=3, learning_rate='auto', metric='cosine', init='pca', perplexity=1).fit_transform(dataset)

In [25]:
tree = KDTree(X_embedded, leaf_size=40)  # creating kd tree
_, ind = tree.query(X_embedded, k=K+1)# quering nearest items
search = SearchSimilarModels()
search.calc_map(ind, label)

0.8068359375

In [48]:
cosine_tree = methods.KDTree(X_embedded, 0)
ind = cosine_tree.search(X_embedded)
search.calc_map(ind, label)

0.878408203125

In [27]:
cosine_knn = methods.DistributedCosineKnn(K)
ind, _ = cosine_knn.fit(X_embedded)
search.calc_map(ind, label)

0.828288302951389

In [28]:
hnsw = methods.HNSW(X_embedded)
ind = hnsw.search(X_embedded, X_embedded, K+1)
search.calc_map(ind, label)

0.826755099826389

# Нормализация

In [49]:
from sklearn.preprocessing import Normalizer

In [50]:
transformer = Normalizer().fit(dataset)  # fit does nothing.

In [51]:
data_norm_sklearn = transformer.transform(dataset)

In [52]:
X_embedded_norm_sklearn = TSNE(n_components=3, learning_rate='auto', metric='cosine', init='pca', perplexity=1).fit_transform(
    data_norm_sklearn)

In [53]:
tree = KDTree(X_embedded_norm_sklearn, leaf_size=40)  # creating kd tree
_, ind = tree.query(X_embedded_norm_sklearn, k=K+1)# quering nearest items
search = SearchSimilarModels()
search.calc_map(ind, label)

0.8111040581597223

In [54]:
cosine_tree = methods.KDTree(X_embedded_norm_sklearn, 0)
ind = cosine_tree.search(X_embedded_norm_sklearn)
search.calc_map(ind, label)

0.8648779296874999

In [55]:
cosine_knn = methods.DistributedCosineKnn(K)
ind, _ = cosine_knn.fit(X_embedded_norm_sklearn)
search.calc_map(ind, label)

0.8233203124999999

In [56]:
hnsw = methods.HNSW(X_embedded_norm)
ind = hnsw.search(X_embedded_norm, X_embedded_norm, K+1)
search.calc_map(ind, label)

0.8050092230902777